In [1]:
import json
#import input file
f = open("annotations/instances_val2017.json",)
q = json.load(f)

In [2]:
q.keys()

dict_keys(['info', 'licenses', 'images', 'annotations', 'categories'])

In [15]:

#load image data, image data contains img ID,file Name, URL, height, width and date captured
 #'coco_url': 'http://images.cocodataset.org/val2017/000000397133.jpg',
 #'height': 427,
 #'width': 640,
 #'date_captured': '2013-11-14 17:02:52',
 #'flickr_url': 'http://farm7.staticflickr.com/6116/6255196340_da26cf2c9e_z.jpg',
 #'id': 397133}
imgDict = {}
for im in q['images']:
    imgDict[im['id']] = im

In [28]:
q['categories']

[{'supercategory': 'person', 'id': 1, 'name': 'person'},
 {'supercategory': 'vehicle', 'id': 2, 'name': 'bicycle'},
 {'supercategory': 'vehicle', 'id': 3, 'name': 'car'},
 {'supercategory': 'vehicle', 'id': 4, 'name': 'motorcycle'},
 {'supercategory': 'vehicle', 'id': 5, 'name': 'airplane'},
 {'supercategory': 'vehicle', 'id': 6, 'name': 'bus'},
 {'supercategory': 'vehicle', 'id': 7, 'name': 'train'},
 {'supercategory': 'vehicle', 'id': 8, 'name': 'truck'},
 {'supercategory': 'vehicle', 'id': 9, 'name': 'boat'},
 {'supercategory': 'outdoor', 'id': 10, 'name': 'traffic light'},
 {'supercategory': 'outdoor', 'id': 11, 'name': 'fire hydrant'},
 {'supercategory': 'outdoor', 'id': 13, 'name': 'stop sign'},
 {'supercategory': 'outdoor', 'id': 14, 'name': 'parking meter'},
 {'supercategory': 'outdoor', 'id': 15, 'name': 'bench'},
 {'supercategory': 'animal', 'id': 16, 'name': 'bird'},
 {'supercategory': 'animal', 'id': 17, 'name': 'cat'},
 {'supercategory': 'animal', 'id': 18, 'name': 'dog'},

In [13]:
#generate CategoryId to category Name  mapping 
#catDict contains SuperCategory mapping
#lowercatDict contains sub category mapping 
catDict = {}
lowercatDict = {}
supercatDict = set()
allcat = set()
for cat in q['categories']:
    catDict[cat['id']] = cat['supercategory']
    lowercatDict[cat['id']] = cat['name']
    supercatDict.add(cat['supercategory'])
    allcat.add(cat['name'])
allcat
# lowercatDict

{'airplane',
 'apple',
 'backpack',
 'banana',
 'baseball bat',
 'baseball glove',
 'bear',
 'bed',
 'bench',
 'bicycle',
 'bird',
 'boat',
 'book',
 'bottle',
 'bowl',
 'broccoli',
 'bus',
 'cake',
 'car',
 'carrot',
 'cat',
 'cell phone',
 'chair',
 'clock',
 'couch',
 'cow',
 'cup',
 'dining table',
 'dog',
 'donut',
 'elephant',
 'fire hydrant',
 'fork',
 'frisbee',
 'giraffe',
 'hair drier',
 'handbag',
 'horse',
 'hot dog',
 'keyboard',
 'kite',
 'knife',
 'laptop',
 'microwave',
 'motorcycle',
 'mouse',
 'orange',
 'oven',
 'parking meter',
 'person',
 'pizza',
 'potted plant',
 'refrigerator',
 'remote',
 'sandwich',
 'scissors',
 'sheep',
 'sink',
 'skateboard',
 'skis',
 'snowboard',
 'spoon',
 'sports ball',
 'stop sign',
 'suitcase',
 'surfboard',
 'teddy bear',
 'tennis racket',
 'tie',
 'toaster',
 'toilet',
 'toothbrush',
 'traffic light',
 'train',
 'truck',
 'tv',
 'umbrella',
 'vase',
 'wine glass',
 'zebra'}

In [32]:
#first task -- Generate data for polygon -- for each Super category pick 1 image
# this generate results for 1 item in an Image

count = 0
AllTasks = []
countDict = {}
#id_generator -- random hash generator for id used in labelStudio for each completion
def id_generator(size=6, chars=string.ascii_uppercase + string.digits):
    return ''.join(random.choice(chars) for _ in range(size))

#Iterate over all annotations/
for an in q['annotations']:
    # is crowd flags show was it annotated by corwdsourcing user or by expert, 0 means by an expert
    if an['iscrowd'] == 0 and an['area'] >= 15000:
        if count > 96:
            break
        # trying to make data equally distributed over all categories, each category has at least 8 images
        if catDict[an['category_id']] in countDict:
            if countDict[catDict[an['category_id']]] >= 8:
                continue
            countDict[catDict[an['category_id']]] = countDict[catDict[an['category_id']]] + 1
        else:
            countDict[catDict[an['category_id']]] = 1
        #get img data from img Dict, based on img id given in annotation data
        myImg = imgDict[an['image_id']]
        
        # if len(an['segmentation']) <= 1:
            # print(len(an['segmentation']))
            # print(an)
            # print(myImg) 
            # continue
        count = count + 1        
        # create task data based on label studio format
        task = {}
        task['text'] = myImg['coco_url']
        task['layout_id'] = 8
        task['groundTruth'] = " "
        task['format_type'] = 1
        task['batch_id'] = 3
        task['description'] = " "
        task['id'] = count

        comData = {}    
        comData['lead_time'] = 3.821
        comData['result'] = []
        comData['user'] = 0

        # for points in an['segmentation'][0]:
        
        pointsWithLabel = {}
        for segPoints in an['segmentation']:
            myre = {}
            myPoints = []
            x,y = 0,0
            #label studio deals with point mapping in % format, annotation data contain x,y coordinates
            # converting x,y coordinates to % form
            for i,p in enumerate(segPoints):
                if i%2 == 0:
                    x = ((p / myImg['width']) * 100)
                    # print(i,p,((p / myImg['width']) * 100))
                else:
                    y = ((p / myImg['height']) * 100)
                    # print(i,p,((p / myImg['height']) * 100))
                    myPoints.append([x,y])
            # creating result data according labelstudio format
            pointsWithLabel['points'] = myPoints
            pointsWithLabel['polygonlabels'] = [catDict[an['category_id']]]
            myre['value'] = pointsWithLabel
            myre['original_width'] = myImg['width']
            myre['original_height'] = myImg['height']
            myre['image_rotation'] = 0
            myre['from_name'] = 'label'
            myre['to_name'] = 'image'
            myre['type'] = 'polygonlabels'
            myre['id'] = id_generator(7)    
            # print("\n")
            # print(segPoints)
            # print(myre)    
            comData['result'].append(myre)
            break
        #generating complations in labelstudio format
        completion = {}
        completion['task_id'] =count
        completion['user_id'] = 0
        completion['data'] = json.dumps(comData)
        completion['completed_at'] = 1616101190
        completion['batch_id'] = 3
        completion['was_skipped'] = 0
        task['completion'] =  completion
        AllTasks.append(task)
        # break

todump = {}
todump["tasks"] = AllTasks
# print(json.dumps(todump))
with open('PolyGon2.json', 'w') as outfile:
    json.dump(todump, outfile)
print(count, "Done")


96 Done


In [30]:
#2nd task -- Generate data for Rectangle  -- for each Super category pick 1 image
# this generate results for 1 item in an Image

import string
import random
def id_generator(size=6, chars=string.ascii_uppercase + string.digits):
    return ''.join(random.choice(chars) for _ in range(size))
    
count = 0
AllTasks = []
countDict = {}

for an in q['annotations']:
    if an['iscrowd'] == 0:
        #Skip first 96 annotations as they were used in previos task
        if count > 96 * 2:
            break
        if count < 96:
            if catDict[an['category_id']] in countDict:
                if  countDict[catDict[an['category_id']]] >= 8:
                    continue
                countDict[catDict[an['category_id']]] = countDict[catDict[an['category_id']]] + 1
            else:
                countDict[catDict[an['category_id']]] = 1
            count = count + 1  
            continue
        else:
            if an['area'] < 20000:
                continue
            if catDict[an['category_id']] in countDict:
                if  countDict[catDict[an['category_id']]] >= 16:
                    continue
                countDict[catDict[an['category_id']]] = countDict[catDict[an['category_id']]] + 1
            else:
                countDict[catDict[an['category_id']]] = 1
        # print(json.dumps(myImg, indent=2))
        # print("\n")
        # print(json.dumps(an, indent=2))
        # if len(an['segmentation']) <= 1:
            # print(len(an['segmentation']))
            # print(an)
            # print(myImg) 
            # continue
        count = count + 1  
        myImg = imgDict[an['image_id']]
        task = {}
        task['text'] = myImg['coco_url']
        task['layout_id'] = 5
        task['groundTruth'] = " "
        task['format_type'] = 1
        task['batch_id'] = 2
        task['description'] = " "
        task['id'] = count

        comData = {}    
        comData['lead_time'] = 3.821
        comData['result'] = []
        comData['user'] = 0

        # for points in an['segmentation'][0]:
        
        pointsWithLabel = {}
        # for segPoints in an['bbox']:
        myre = {}
        myPoints = {}
        sx,lx,sy,ly = -1,-1,-1,-1
        # for i,p in enumerate(segPoints):
            # if i % 2 == 0:
                # x = ((p / myImg['width']) * 100)
                # if sx == -1 or x < sx:
                #     sx = x
                # if lx == -1 or x > lx:
                #     lx = x
                # # print(i,p,((p / myImg['width']) * 100))
            # else:
                # y = ((p / myImg['height']) * 100)
                # if sy == -1 or y < sy:
                #     sy = y
                # if ly == -1 or y > ly:
                #     ly = y
                # print(i,p,((p / myImg['height']) * 100))
                # myPoints.append([x,y])
                
        # myPoints["x"] = sx
        # myPoints["y"] = sy
        # myPoints["width"] = lx - sx
        # myPoints["heigth"] = ly - sy
        # print(an['bbox'])
        
        #generate bounding box data in label studio format
        myPoints["x"] = ((an['bbox'][0] / myImg['width']) * 100)
        myPoints["y"] = ((an['bbox'][1] / myImg['height']) * 100)
        myPoints["width"] = ((an['bbox'][2] / myImg['width']) * 100)
        myPoints["height"] = ((an['bbox'][3] / myImg['height']) * 100)
        # print(json.dumps(myPoints, indent=2))
        myPoints['rotation'] = 0
        myPoints['rectanglelabels'] = [catDict[an['category_id']]]    

        myre['original_width'] = myImg['width']
        myre['original_height'] = myImg['height']
        myre['image_rotation'] = 0
        myre['value'] = myPoints
        myre['id'] = id_generator(7)  
        myre['from_name'] = 'object'
        myre['to_name'] = 'text'
        myre['type'] = 'rectanglelabels'  
        # print("\n")
        # print(segPoints)
        # print(myre)    
        comData['result'].append(myre)
        # break
        completion = {}
        completion['task_id'] =count
        completion['user_id'] = 0
        completion['data'] = json.dumps(comData)
        completion['completed_at'] = 1616101190
        completion['batch_id'] = 2
        completion['was_skipped'] = 0
        task['completion'] =  completion
        AllTasks.append(task)
        # break

todump = {}
todump["tasks"] = AllTasks
# print(json.dumps(todump))
with open('Rectangle2.json', 'w') as outfile:
    json.dump(todump, outfile)
# print(json.dumps(AllTasks, indent=2))
print(count-96, "Done")


96 Done


In [27]:
#3rd task -- Generate data for MCQ for images


import string
import random
def id_generator(size=6, chars=string.ascii_uppercase + string.digits):
    return ''.join(random.choice(chars) for _ in range(size))
    
count = 0
AllTasks = {}
countDict = {}

for an in q['annotations']:
    if an['iscrowd'] == 0:
        # if len(AllTasks) > 100:
            # break
        # skip 192 annotations as they were used in previos 2 tasks
        if count < 96 * 2:
            # if lowercatDict[an['category_id']] in countDict:
            #     if  countDict[lowercatDict[an['category_id']]] >= 16:
            #         continue
            #     countDict[lowercatDict[an['category_id']]] = countDict[lowercatDict[an['category_id']]] + 1
            # else:
            #     countDict[lowercatDict[an['category_id']]] = 1
            count = count + 1  
            continue
        # else:
        #     if len(AllTasks) > 100:
        #         continue
        #     if lowercatDict[an['category_id']] in countDict:
        #         if  countDict[lowercatDict[an['category_id']]] >= 8:
        #             continue
        #         countDict[lowercatDict[an['category_id']]] = countDict[lowercatDict[an['category_id']]] + 1
        #     else:
        #         countDict[lowercatDict[an['category_id']]] = 1
        if an['area'] < 10000:
            continue

        count = count + 1
        if an['image_id'] not in AllTasks:
            # dont add more new Images, only find data for already added images
            if len(AllTasks) > 100:
                continue
            myImg = imgDict[an['image_id']]
            task = {}
            task['text'] = myImg['coco_url']
            task['layout_id'] = 9
            task['groundTruth'] = " "
            task['format_type'] = 1
            task['batch_id'] = 4
            task['description'] = " "
            task['id'] = count

            comData = {}    
            comData['lead_time'] = 3.821
            comData['result'] = []
            comData['user'] = 0

            choices = {} 
            choices['choices'] = set()
            choices['choices'].add(lowercatDict[an['category_id']])
            myre = {}
            myre['value'] = choices
            myre['id'] = id_generator(7)  
            myre['from_name'] = 'choice'
            myre['to_name'] = 'image'
            myre['type'] = 'choices'  
            comData['result'].append(myre)

            completion = {}
            completion['task_id'] =count
            completion['user_id'] = 0
            completion['data'] = comData #json.dumps(comData)
            completion['completed_at'] = 1616101190
            completion['batch_id'] = 4
            completion['was_skipped'] = 0
            task['completion'] =  completion
            AllTasks[an['image_id']] = task
            # print(task)            
        else:
            task = AllTasks[an['image_id']]
            task['completion']['data']['result'][0]['value']['choices'].add(lowercatDict[an['category_id']])
            # print(task)
            # break

# {"lead_time": 15.143, "result": [{"value": {"choices": ["Airbus", "Car"]}, "id": "ZeDyaKp3ve", "from_name": "choice", "to_name": "image", "type": "choices"}], "user": "2", "created_at": 1618832764}

todump = {}
NewAllTask = []

for taskid in AllTasks:
    task = AllTasks[taskid]
    choices = []
    # for choice in task['completion']['data']['result'][0]['value']['choices']:
        # choices.append(choice)
    task['completion']['data']['result'][0]['value']['choices'] = list(task['completion']['data']['result'][0]['value']['choices'])
    task['completion']['data'] = json.dumps(task['completion']['data'])
    NewAllTask.append(task)
todump["tasks"] = NewAllTask
# print(json.dumps(todump))
with open('ImageMC.json', 'w') as outfile:
    json.dump(todump, outfile)
# print(json.dumps(AllTasks, indent=2))
print(count-96*2, "Done")


36143 Done


In [62]:
#same as first task -- Generate data for polygon -- 
# this generate results for all annotated item in an Image

count = 0
AllTasks = {}
countDict = {}
def id_generator(size=6, chars=string.ascii_uppercase + string.digits):
    return ''.join(random.choice(chars) for _ in range(size))

for an in q['annotations']:
    if an['iscrowd'] == 0 and an['area'] >= 10000:
        # if count > 96:
            # break
        
        # if len(an['segmentation']) <= 1:
            # print(len(an['segmentation']))
            # print(an)
            # print(myImg) 
            # continue
        # print(an['image_id'])
        if an['image_id'] not in AllTasks:   
            # if "000000182417" not in imgDict[an['image_id']]['coco_url']:
            #     continue
            # if catDict[an['category_id']] in countDict:
            #     if countDict[catDict[an['category_id']]] >= 8:
            #         continue
            #     countDict[catDict[an['category_id']]] = countDict[catDict[an['category_id']]] + 1
            # else:
            #     countDict[catDict[an['category_id']]] = 1
            
            myImg = imgDict[an['image_id']]            
            # if len(AllTasks) > 100:
            #     continue     
            task = {}
            task['text'] = myImg['coco_url']
            task['layout_id'] = 8
            task['groundTruth'] = " "
            task['format_type'] = 1
            task['batch_id'] = 3
            task['description'] = " "
            task['id'] = count

            comData = {}    
            comData['lead_time'] = 3.821
            comData['result'] = []
            comData['user'] = 0

            # for points in an['segmentation'][0]:
            
            pointsWithLabel = {}
            for segPoints in an['segmentation']:
                myre = {}
                myPoints = []
                x,y = 0,0
                for i,p in enumerate(segPoints):
                    if i%2 == 0:
                        x = ((p / myImg['width']) * 100)
                        # print(i,p,((p / myImg['width']) * 100))
                    else:
                        y = ((p / myImg['height']) * 100)
                        # print(i,p,((p / myImg['height']) * 100))
                        myPoints.append([x,y])
                pointsWithLabel['points'] = myPoints
                pointsWithLabel['polygonlabels'] = [catDict[an['category_id']]]
                myre['value'] = pointsWithLabel
                myre['original_width'] = myImg['width']
                myre['original_height'] = myImg['height']
                myre['image_rotation'] = 0
                myre['from_name'] = 'label'
                myre['to_name'] = 'image'
                myre['type'] = 'polygonlabels'
                myre['id'] = id_generator(7)    
                # print("\n")
                # print(segPoints)
                # print(myre)    
                comData['result'].append(myre)
                break
            completion = {}
            completion['task_id'] =count
            completion['user_id'] = 0
            completion['data'] = comData #json.dumps(comData)
            completion['completed_at'] = 1616101190
            completion['batch_id'] = 3
            completion['was_skipped'] = 0
            task['completion'] =  completion
            # AllTasks.append(task)
            AllTasks[an['image_id']] = task
            count = count + 1
        else:
            task = AllTasks[an['image_id']]
            pointsWithLabel = {}
            myImg = imgDict[an['image_id']]
            for segPoints in an['segmentation']:
                myre = {}
                myPoints = []
                x,y = 0,0
                for i,p in enumerate(segPoints):
                    if i%2 == 0:
                        x = ((p / myImg['width']) * 100)
                        # print(i,p,((p / myImg['width']) * 100))
                    else:
                        y = ((p / myImg['height']) * 100)
                        # print(i,p,((p / myImg['height']) * 100))
                        myPoints.append([x,y])
                pointsWithLabel['points'] = myPoints
                pointsWithLabel['polygonlabels'] = [catDict[an['category_id']]]
                myre['value'] = pointsWithLabel
                myre['original_width'] = myImg['width']
                myre['original_height'] = myImg['height']
                myre['image_rotation'] = 0
                myre['from_name'] = 'label'
                myre['to_name'] = 'image'
                myre['type'] = 'polygonlabels'
                myre['id'] = id_generator(7)
                # print("same imge found")
                # print(segPoints)
                break    
                # print(myre)    
                # comData['result'].append(myre)
            task['completion']['data']['result'].append(myre)
            # print(task)    
            # break

todump = {}
todump = {}
NewAllTask = []

for taskid in AllTasks:
    task = AllTasks[taskid]
    if len(task['completion']['data']['result']) >= 2 and len(task['completion']['data']['result']) <= 4:
        task['completion']['data'] = json.dumps(task['completion']['data'])
        NewAllTask.append(task)
        if len(NewAllTask) > 100:
            break
todump["tasks"] = NewAllTask

# print(json.dumps(todump))
with open('PolyGon3.json', 'w') as outfile:
    json.dump(todump, outfile)
print(count, "Done")


3996 Done


In [33]:
{"lead_time": 121.449, "result": [{"value": {"points": [[9.644670050761421, 83.2], [9.644670050761421, 70.4], [22.588832487309645, 70], [33.756345177664976, 71], [27.918781725888326, 85.2]], "polygonlabels": ["animal"]}, "original_width": 394, "original_height": 500, "image_rotation": 0, "id": "V2qaNk8Ezn", "from_name": "label", "to_name": "image", "type": "polygonlabels"}, {"value": {"points": [[64.46700507614213, 74], [76.14213197969544, 51.2], [85.0253807106599, 54.8], [85.0253807106599, 65.2], [76.6497461928934, 68.4]], "polygonlabels": ["appliance"]}, "original_width": 394, "original_height": 500, "image_rotation": 0, "id": "glh15uZnJu", "from_name": "label", "to_name": "image", "type": "polygonlabels"}], "user": "2", "created_at": 1620039285}

{'lead_time': 121.449,
 'result': [{'value': {'points': [[9.644670050761421, 83.2],
     [9.644670050761421, 70.4],
     [22.588832487309645, 70],
     [33.756345177664976, 71],
     [27.918781725888326, 85.2]],
    'polygonlabels': ['animal']},
   'original_width': 394,
   'original_height': 500,
   'image_rotation': 0,
   'id': 'V2qaNk8Ezn',
   'from_name': 'label',
   'to_name': 'image',
   'type': 'polygonlabels'},
  {'value': {'points': [[64.46700507614213, 74],
     [76.14213197969544, 51.2],
     [85.0253807106599, 54.8],
     [85.0253807106599, 65.2],
     [76.6497461928934, 68.4]],
    'polygonlabels': ['appliance']},
   'original_width': 394,
   'original_height': 500,
   'image_rotation': 0,
   'id': 'glh15uZnJu',
   'from_name': 'label',
   'to_name': 'image',
   'type': 'polygonlabels'}],
 'user': '2',
 'created_at': 1620039285}

In [63]:
q['images'][0]

{'license': 4,
 'file_name': '000000397133.jpg',
 'coco_url': 'http://images.cocodataset.org/val2017/000000397133.jpg',
 'height': 427,
 'width': 640,
 'date_captured': '2013-11-14 17:02:52',
 'flickr_url': 'http://farm7.staticflickr.com/6116/6255196340_da26cf2c9e_z.jpg',
 'id': 397133}